# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [14]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

## Read the CSV and Perform Basic Data Cleaning

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
binary_encoded_df = pd.get_dummies(df, columns=['home_ownership','issue_d','verification_status','pymnt_plan','hardship_flag','initial_list_status','next_pymnt_d','application_type','debt_settlement_flag'])
binary_encoded_df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Verified,pymnt_plan_n,hardship_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,1,1,0,1,0,1,1,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,1,0,1,0,1,1,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,1,0,1,0,1,1,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,1,0,1,0,1,1,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,1,1,0,1,0,1,1,0,1


## Split the Data into Training and Testing

In [8]:
# Create our features
X = binary_encoded_df.drop(columns='loan_status')

# Create our target
y = binary_encoded_df.loan_status


In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,hardship_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,1.0,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.0,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [10]:
# Check the balance of our target values
y = pd.DataFrame(y)
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [15]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train,X_test, y_train,y_test = train_test_split(X, y,random_state=1,stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [16]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [17]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [18]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [34]:
# Resample the training data with the BalancedRandomForestClassifier
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
model.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [22]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.5

In [23]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   87,     0],
       [17118,     0]], dtype=int64)

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      1.00      0.00      0.01      0.00      0.00        87
   low_risk       0.00      0.00      1.00      0.00      0.00      0.00     17118

avg / total       0.00      0.01      0.99      0.00      0.00      0.00     17205



In [25]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_
importances_sorted = sorted(zip(model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.06804680638504733, 'total_rec_prncp'),
 (0.06434017293143314, 'total_rec_int'),
 (0.060378004680789424, 'total_pymnt_inv'),
 (0.05930573849268042, 'total_pymnt'),
 (0.054956467459570855, 'last_pymnt_amnt'),
 (0.027800485303735408, 'int_rate'),
 (0.01917855453181943, 'issue_d_Jan-2019'),
 (0.019013087318359594, 'dti'),
 (0.01699234139402735, 'max_bal_bc'),
 (0.01578270714837746, 'installment'),
 (0.015756547532492227, 'out_prncp_inv'),
 (0.015472698716911381, 'annual_inc'),
 (0.01545301014524211, 'bc_util'),
 (0.015407941289729957, 'revol_bal'),
 (0.015397276787527395, 'out_prncp'),
 (0.015329644716527119, 'mths_since_recent_inq'),
 (0.015024775327520282, 'il_util'),
 (0.014744809812277123, 'mo_sin_old_rev_tl_op'),
 (0.014579571477622642, 'mo_sin_old_il_acct'),
 (0.014117733491059012, 'tot_hi_cred_lim'),
 (0.014060529751245639, 'issue_d_Mar-2019'),
 (0.013747778338312403, 'avg_cur_bal'),
 (0.013339031897560534, 'mths_since_rcnt_il'),
 (0.013302934359422336, 'total_rev_hi_lim'),
 (0.

### Easy Ensemble Classifier

In [38]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model2= EasyEnsembleClassifier(n_estimators=100, random_state=1)
model2.fit(X_train_scaled, y_train)

AttributeError: 'EasyEnsembleClassifier' object has no attribute 'n_features_in_'

In [33]:
# Calculated the balanced accuracy score
y_pred = model2.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

AttributeError: 'EasyEnsembleClassifier' object has no attribute 'estimators_'

In [35]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   87,     0],
       [17118,     0]], dtype=int64)

In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      1.00      0.00      0.01      0.00      0.00        87
   low_risk       0.00      0.00      1.00      0.00      0.00      0.00     17118

avg / total       0.00      0.01      0.99      0.00      0.00      0.00     17205

